# Softmax Regression

In [1]:
import numpy as np
import scipy.sparse
import pickle
from sklearn.linear_model import LogisticRegression
import time
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import accuracy_score

*Softmax Regression* (synonyms: *Multinomial Logistic*, *Maximum Entropy Classifier*, or just *Multi-class Logistic Regression*) is a generalization of logistic regression that we can use for multi-class classification (under the assumption that the classes are  mutually exclusive). 

The schematic diagram of (standard) *Logistic Regression* model in binary classification tasks.

![](logistic_regression_schematic_2.png)

### The schematic diagram for Softmax Regression

![](softmax_schematic_1.png)

In *Softmax Regression*, we replace the sigmoid logistic function by the so-called *softmax* function $\phi_{softmax}(\cdot)$.

$$P(y=i \mid z^{(i)}) = \phi_{softmax}(z^{(i)}) = \frac{e^{z^{(i)}}}{\sum_{j=0}^{k} e^{z_{j}^{(i)}}},$$

where we define the net input *z* as 

$$z = w_1x_1 + ... + w_mx_m  + b= \sum_{l=0}^{m} w_l x_l + b= \mathbf{w}^T\mathbf{x} + b.$$ 

(**w** is the weight vector, $\mathbf{x}$ is the feature vector of 1 training sample, and $b$ is the bias unit.)   
Now, this softmax function computes the probability that this training sample $\mathbf{x}^{(i)}$ belongs to class $j$ given the weight and net input $z^{(i)}$. So, we compute the probability $p(y = j \mid \mathbf{x^{(i)}; w}_j)$ for each class label in  $j = 1, \ldots, k.$. Note the normalization term in the denominator which causes these class probabilities to sum up to one.

## Helper functions 

We are assuming that the input is vectorised and in the form of X = [samples,features]
and the target labels are y with actual labels 0,1,..9. and the splits are trainX, trainY, testX and testY.

First of all, we need to define a function for the softmax activation

Now, it's time to compute the softmax activation that we discussed earlier:

$$\phi_{softmax}(z) = \frac{e^{z}}{\sum_{j=0}^{k} e^{z_{k}}}.$$

In [23]:
def softmax(x):
    ex = np.exp(x-np.max(x))
    deno = np.sum(ex, axis=1).reshape(-1,1)
    a = np.divide(ex,deno)
    return np.divide(ex, deno)
    # write the code for softmax function

In [3]:
# function to load one batch
def getData(file) :
    name = './cifar-10-batches-py/'+file
    with open(name, 'rb') as fo:
        data = pickle.load(fo, encoding="bytes")
    return data# function to load one batch


In [4]:

# loading train-validation data
X = []
y = []
for i in range(5) :
    filename = "data_batch_"+str(i+1)
    data = getData(filename)
    X.append(data[b'data'])
    y.append(data[b'labels'])


In [5]:
# input of size (50000, 3072) and output of size (50000,1)
X = np.array(X).reshape(-1,3072)
X = np.divide(X,255)
y = np.array(y).reshape(50000)


In [27]:
# loading test data
Xtest = getData("test_batch")[b'data']
Ytest = np.array(getData("test_batch")[b'labels'])
Xtest = np.divide(Xtest,255)


In [7]:
label_names = [lbl.decode("utf-8") for lbl in getData("batches.meta")[b'label_names']]

[xtrain,xtest,ytrain,ytest] = train_test_split(X,y,random_state=42,train_size=0.8)


/home/meenu/.local/lib/python3.5/site-packages/sklearn/model_selection/_split.py:2026: FutureWarning: From version 0.21, test_size will always complement train_size unless both are specified.
  FutureWarning)


We need to apply one-hot encoding to the target labels.

In [40]:
def oneHotIt(Y):
    yHot = np.zeros((Y.shape[0],10))
#     print(yHot.shape)
    index = 0
    for i in Y :
        yHot[index][i] = 1
        index += 1
    return yHot
    # write the code to convert Y into one-hot encoded labels.


Softmax uses Cross-entropy loss to learn the weights.

$$J(\mathbf{W}; \mathbf{b}) = \frac{1}{n} \sum_{i=1}^{n} H(T_i, O_i),$$

which is the average of all cross-entropies over our $n$ training samples. The cross-entropy  function is defined as

$$H(T_i, O_i) = -\sum_m T_i \cdot log(O_i).$$

Here the $T$ stands for "target" (i.e., the *true* class labels) and the $O$ stands for output -- the computed *probability* via softmax; **not** the predicted class label.

In order to learn our softmax model -- determining the weight coefficients -- via gradient descent, we then need to compute the derivative of the loss,

$$\nabla \mathbf{w}_j \, J(\mathbf{W}; \mathbf{b}).$$

we won't walk through the tedious details here, but this cost derivative turns out to be simply:

$$\nabla \mathbf{w}_j \, J(\mathbf{W}; \mathbf{b}) = \frac{1}{n} \sum^{n}_{i=0} \big[\mathbf{x}^{(i)}\ \big(O_i - T_i \big) \big]$$

We can then use the cost derivate to update the weights in opposite direction of the cost gradient with learning rate $\eta$:

$$\mathbf{w}_j := \mathbf{w}_j - \eta \nabla \mathbf{w}_j \, J(\mathbf{W}; \mathbf{b})$$ 

for each class $$j \in \{0, 1, ..., k\}$$

(note that $\mathbf{w}_j$ is the weight vector for the class $y=j$), and we update the bias units


$$\mathbf{b}_j := \mathbf{b}_j   - \eta \bigg[ \frac{1}{n} \sum^{n}_{i=0} \big(O_i - T_i  \big) \bigg].$$ 
 


As a penalty against complexity, an approach to reduce the variance of our model and decrease the degree of overfitting by adding additional bias, we can further add a regularization term such as the L2 term with the regularization parameter $\lambda$:
    
L2:        $\frac{\lambda}{2} ||\mathbf{w}||_{2}^{2}$, 

where 

$$||\mathbf{w}||_{2}^{2} = \sum^{m}_{l=0} \sum^{k}_{j=0} w_{i, j}$$

so that our cost function becomes

$$J(\mathbf{W}; \mathbf{b}) = \frac{1}{n} \sum_{i=1}^{n} H(T_i, O_i) + \frac{\lambda}{2} ||\mathbf{w}||_{2}^{2}$$

and we define the "regularized" weight update as

$$\mathbf{w}_j := \mathbf{w}_j -  \eta \big[\nabla \mathbf{w}_j \, J(\mathbf{W}) + \lambda \mathbf{w}_j \big].$$

(Please note that we don't regularize the bias term.)

This function getLoss takes in the weights w, the input x, they targets y and a regularising coefficient lam.

In [25]:
def getLoss(w,x,y,lam=0):
    m = x.shape[0]#First we get the number of training examples
    y_mat = oneHotIt(y) #Next we convert the integer class coding into a one-hot representation using the oneHotit function
    y_true = np.argmax(np.array(y),axis=1)
    scores = np.matmul(x,w)#Then we compute raw class scores given our input and current weights (w*x)
    prob = softmax(scores) #Next we perform a softmax on these scores to get their probabilities
    sample = 0
    totalLoss = 0
    
    for category in y_true :
        totalLoss += (-np.log(prob[sample][category]))
        sample += 1 
    loss = totalLoss/sample #We then find the loss of the probabilities
    grad = (-1 / m) * np.dot(x.T,(y_mat - prob)) + lam*w #And compute the gradient for that loss
    return loss,grad

In [10]:
def getProbsAndPreds(someX):
    probs = softmax(np.dot(someX,w))
    preds = np.argmax(probs,axis=1)
    return probs,preds

In [42]:
trainX = xtrain
trainY = ytrain.reshape(-1,1)
w = np.zeros([trainX.shape[1],len(np.unique(trainY))]) ## initialise the weights
lam = 0
iterations = 100 ## mention the number of iterations 
learningRate = 1e-2
losses = []
for i in range(0,iterations):
    loss,grad = getLoss(w, trainX, trainY ,lam)
    losses.append(loss)
    w = w - (learningRate * grad)

In [43]:
def getAccuracy(someX,someY):
    prob,prede = getProbsAndPreds(someX)
    accuracy = sum(prede == someY)/(float(len(someY)))
    return accuracy

In [44]:
print('Training Accuracy: ', getAccuracy(xtrain,ytrain))
print('Validation Accuracy: ', getAccuracy(xtest,ytest))

print('Test Accuracy: ', getAccuracy(Xtest,Ytest))

Training Accuracy:  0.323725
Validation Accuracy:  0.316
Test Accuracy:  0.3202


In [ ]:
# The accuracy of both softmax regression and one vs rest for 10 classifiers have less accuracy when compared to 3 
# class classifiers. this is because the model is not complex enough and it underfits the given data